## Grock sentiment test

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from pydantic import BaseModel, Field
from dataclasses import dataclass
from pydantic_ai import Agent, ModelRetry, RunContext
from typing import Optional
import os
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openrouter import OpenRouterProvider

## Extraction of data from text

In [5]:
class TopicFilter(BaseModel):
    """Whether this is related to Trump/Putin meeting"""
    topic_match: bool = Field(description = "Whether this news match the topic (Yes or No)")
    explanation: Optional[str] = Field(description = "explanation after of Yes/No answer")

In [6]:
@dataclass
class Deps:
    """Dependencies"""

In [7]:
model = OpenAIModel(
    'x-ai/grok-4',
    provider=OpenRouterProvider(api_key=os.getenv("OPENROUTER_API_KEY")),
)


topic_filter_agent = Agent[Deps, TopicFilter](
    model,
    output_type=TopicFilter,
    retries=3,
    system_prompt=(
        '''
        ## Role
        You are an AI classifier specialized in analyzing news tweets.
        Your task is to determine if a given tweet from one of the following mass media accounts @BBCBreaking, @Reuters, @AP, or @FoxNews
        is related to the outcome of the meeting between Russian President Vladimir Putin and U.S. President Donald Trump,
        which occurred on August 15, 2025, at Joint Base Elmendorf-Richardson in Anchorage, Alaska.
        This summit focused on topics like a potential ceasefire in the Russia-Ukraine war.
        ## Input
            You will receive the text of a single tweet as input. Assume it is from one of the listed accounts.
        ## Output
            Respond with exactly one of the following:
                "Yes" if the tweet is related to the meeting's outcome (e.g., decisions, agreements, statements, reactions, or results from the summit). Use semantic and contextual analysis to infer relevance, even if the date, location, or explicit terms like "summit" or "meeting" are omitted. For example, references to Putin-Trump talks, Russia-US diplomacy, Ukraine ceasefire announcements, or post-meeting developments could qualify.
                "No" if the tweet is unrelated (e.g., about other events, general news, or topics not tied to this specific meeting's results).

            Provide a brief 1-2 sentence explanation after your Yes/No answer, justifying your decision based on key elements in the tweet. Do not output anything else.
        '''
    ),
)

In [11]:
await topic_filter_agent.run('Trump says he’s "very proud" of "solving six wars in six months"')

AgentRunResult(output=TopicFilter(topic_match=False, explanation="The tweet features a general statement by Trump about solving multiple wars, but it lacks any specific reference to the Putin-Trump meeting, the summit's outcomes, or the Russia-Ukraine ceasefire, making it unrelated."))

In [15]:
await topic_filter_agent.run("All eyes on Washington, and naught but deafening silence from the District's loudest defender")

AgentRunResult(output=TopicFilter(topic_match=False, explanation="The tweet's cryptic reference to silence from a figure in Washington does not mention or imply any connection to the Putin-Trump meeting, its outcomes, or related topics like the Russia-Ukraine ceasefire."))

In [12]:
await topic_filter_agent.run('‘IT’S LIKE CHESS’: Donald J. Trump reveals his game plan for the high-stakes summit with Putin in Alaska, outlining how his negotiations with the Russian leader will differ from those of past presidents — including Bush, Obama, and Biden.')

AgentRunResult(output=TopicFilter(topic_match=False, explanation="The tweet focuses on Trump's revealed game plan and approach for the upcoming summit with Putin in Alaska, which is preparatory in nature and not related to the meeting's outcomes, decisions, or results."))

In [14]:
await topic_filter_agent.run('Trump`s first comments after the meeting: I`m very happy with agreement we`ve reached with Putin, the ceasfire starts tomorrow')

AgentRunResult(output=TopicFilter(topic_match=True, explanation="The tweet directly quotes Trump's positive comments on an agreement reached with Putin, including a ceasefire starting tomorrow, which aligns with the summit's focus on a potential Russia-Ukraine ceasefire."))

In [16]:
class AlignementData(BaseModel):
    """How aligned Russian President Vladimir Putin and U.S. President Donald Trump after the meeting"""
    score: Optional[int] = Field(description = "how aligned are both presidents on a scale from 1 to 10, of None if score is N/A")
    explanation: str = Field(description = "explanation after your score (or N/A)")

In [19]:
topic_sentiment_agent = Agent[Deps, AlignementData](
    model,
    output_type=AlignementData,
    retries=3,
    system_prompt=(
        '''
            # Role
            You are an AI analyzer specialized in evaluating diplomatic outcomes from news tweets.
            Your task is to assess, based on a given tweet from one of the following mass media accounts: @BBCBreaking, @Reuters, @AP, or @FoxNews
            how aligned Russian President Vladimir Putin and U.S. President Donald Trump appear to be after their meeting on August 15, 2025, at Joint Base Elmendorf-Richardson in Anchorage, Alaska.
            This summit focused on topics like a potential ceasefire in the Russia-Ukraine war.
            Alignment refers to the degree of agreement, satisfaction, and positive progress reported or implied in the tweet.

            ## Input
            You will receive the text of a single tweet as input.
            Assume it is from one of the listed accounts and relates to the meeting's outcome.
            Use semantic and contextual analysis to infer alignment, even if details like date, location,
            explicit terms like "summit" or "meeting," or direct quotes are omitted.
            Consider elements such as agreements reached, statements from parties, reactions, tone, and implications for Russia-US relations or the Ukraine conflict.
            
            ## Output
            Respond with a single integer from 1 to 10, where:
            
            - 1: The meeting made no difference; no agreements were reached, and parties remain unaligned or hostile.
            - 5: Moderate progress; some agreements or discussions occurred, but alignment is partial with lingering disagreements.
            - 10: The outcome exceeded expectations; both parties are highly aligned, happy with agreements, and show strong mutual satisfaction.
                        
            Provide a brief 1-2 sentence explanation after your score (or N/A), justifying your decision based on key elements in the tweet. Do not output anything else.
        '''
    ),
)

In [20]:
await topic_sentiment_agent.run("All eyes on Washington, and naught but deafening silence from the District's loudest defender")

AgentRunResult(output=AlignementData(score=1, explanation="The tweet describes deafening silence from the District's loudest defender, interpreted as Trump, who is typically vocal, suggesting the meeting with Putin yielded no agreements or positive outcomes, indicating no alignment or progress."))

In [22]:
await topic_sentiment_agent.run('The meeting in Alaska have anded abruptly, Russian delegation have left without press-conferense. President Trump said: "It`s time to move on to plan B"')

AgentRunResult(output=AlignementData(score=2, explanation="The abrupt ending of the meeting without a press conference and the Russian delegation's departure, coupled with Trump's statement about moving to 'plan B,' indicate a breakdown in talks with no agreements reached, suggesting low alignment and potential hostility."))

In [23]:
await topic_sentiment_agent.run('Trump`s first comments after the meeting: I`m very happy with agreement we`ve reached with Putin, the ceasfire starts tomorrow')

AgentRunResult(output=AlignementData(score=9, explanation="Trump expresses high satisfaction with the agreement reached with Putin, indicating strong alignment and positive progress, including an imminent ceasefire, though Putin's direct reaction is not mentioned."))

# Test cases

@Reuters: Putin and Trump conclude Alaska summit with no progress on Ukraine ceasefire, both sides restate entrenched positions. Alignment Score: 2 Explanation: The tweet indicates no progress and both leaders maintaining their original stances, suggesting minimal alignment.

@AP: Trump-Putin talks in Anchorage end without agreement, with US and Russia blaming each other for stalled Ukraine peace efforts. Alignment Score: 2 Explanation: The lack of agreement and mutual blame reflect a failure to align, pointing to a low alignment score.

@BBCBreaking: Putin and Trump agree to continue talks on Ukraine but fail to finalize ceasefire terms in Alaska meeting. Alignment Score: 5 Explanation: The tweet suggests some willingness to continue discussions, indicating moderate alignment despite no final agreement.

@FoxNews: Anchorage summit sees Putin and Trump commit to working groups on Ukraine, though major differences remain unresolved. Alignment Score: 5 Explanation: Commitment to further talks shows partial alignment, but unresolved differences keep the score moderate.

@Reuters: Trump and Putin reach preliminary agreement on Ukraine ceasefire monitoring in Alaska, positive steps noted by both. Alignment Score: 7 Explanation: A preliminary agreement and positive tone suggest good progress toward alignment, though not fully conclusive.

@AP: Putin and Trump announce joint plan to de-escalate Ukraine conflict post-Anchorage summit, with both expressing cautious optimism. Alignment Score: 7 Explanation: A joint plan and optimism indicate strong progress, but cautious language suggests alignment is not yet maximal.

@BBCBreaking: Trump and Putin hail Alaska summit as breakthrough, agree on major Ukraine ceasefire terms with follow-up talks set. Alignment Score: 9 Explanation: The tweet highlights a major agreement and positive sentiment, indicating high alignment just short of exceeding expectations.

@FoxNews: Anchorage meeting yields historic Putin-Trump deal on Ukraine peace framework, both leaders praise strong partnership. Alignment Score: 9 Explanation: A historic deal and praise for partnership reflect near-maximal alignment, with both sides highly satisfied.

In [24]:
import json

tests_json = '''
[
  {
    "headline": "@Reuters: Putin and Trump conclude Alaska summit with no progress on Ukraine ceasefire, both sides restate entrenched positions.",
    "expected_score": 2
  },
  {
    "headline": "@AP: Trump-Putin talks in Anchorage end without agreement, with US and Russia blaming each other for stalled Ukraine peace efforts.",
    "expected_score": 2
  },
  {
    "headline": "@BBCBreaking: Putin and Trump agree to continue talks on Ukraine but fail to finalize ceasefire terms in Alaska meeting.",
    "expected_score": 5
  },
  {
    "headline": "@FoxNews: Anchorage summit sees Putin and Trump commit to working groups on Ukraine, though major differences remain unresolved.",
    "expected_score": 5
  },
  {
    "headline": "@Reuters: Trump and Putin reach preliminary agreement on Ukraine ceasefire monitoring in Alaska, positive steps noted by both.",
    "expected_score": 7
  },
  {
    "headline": "@AP: Putin and Trump announce joint plan to de-escalate Ukraine conflict post-Anchorage summit, with both expressing cautious optimism.",
    "expected_score": 7
  },
  {
    "headline": "@BBCBreaking: Trump and Trump hail Alaska summit as breakthrough, agree on major Ukraine ceasefire terms with follow-up talks set.",
    "expected_score": 9
  },
  {
    "headline": "@FoxNews: Anchorage meeting yields historic Putin-Trump deal on Ukraine peace framework, both leaders praise strong partnership.",
    "expected_score": 9
  }
]
'''

test_cases = json.loads(tests_json)

In [25]:
test_cases

[{'headline': '@Reuters: Putin and Trump conclude Alaska summit with no progress on Ukraine ceasefire, both sides restate entrenched positions.',
  'expected_score': 2},
 {'headline': '@AP: Trump-Putin talks in Anchorage end without agreement, with US and Russia blaming each other for stalled Ukraine peace efforts.',
  'expected_score': 2},
 {'headline': '@BBCBreaking: Putin and Trump agree to continue talks on Ukraine but fail to finalize ceasefire terms in Alaska meeting.',
  'expected_score': 5},
 {'headline': '@FoxNews: Anchorage summit sees Putin and Trump commit to working groups on Ukraine, though major differences remain unresolved.',
  'expected_score': 5},
 {'headline': '@Reuters: Trump and Putin reach preliminary agreement on Ukraine ceasefire monitoring in Alaska, positive steps noted by both.',
  'expected_score': 7},
 {'headline': '@AP: Putin and Trump announce joint plan to de-escalate Ukraine conflict post-Anchorage summit, with both expressing cautious optimism.',
  'e

In [31]:
import asyncio

max_concurrent = 5
# Semaphore to limit concurrent requests
semaphore = asyncio.Semaphore(max_concurrent)

async def process_single_case_with_limit(test_case: str, index: int):
    """Process a single test case with concurrency limit"""
    async with semaphore:
        try:
            result = await topic_sentiment_agent.run(test_case['headline'])
            return {
                'promt': test_case['headline'],
                'result': result,
                'expected': test_case['expected_score'],
                'success': True,
                'error': None
            }
        except Exception as e:
            return {
                'index': index,
                'promt': test_case['headline'],
                'result': None,
                'success': False,
                'error': str(e)
            }


tasks = [
    process_single_case_with_limit(test_case, index) 
    for index, test_case in enumerate(test_cases)
]
results = await asyncio.gather(*tasks)

In [32]:
results

[{'promt': '@Reuters: Putin and Trump conclude Alaska summit with no progress on Ukraine ceasefire, both sides restate entrenched positions.',
  'result': AgentRunResult(output=AlignementData(score=1, explanation='The tweet indicates no progress on the Ukraine ceasefire with both leaders restating entrenched positions, showing the meeting made no difference and parties remain unaligned.')),
  'expected': 2,
  'success': True,
  'error': None},
 {'promt': '@AP: Trump-Putin talks in Anchorage end without agreement, with US and Russia blaming each other for stalled Ukraine peace efforts.',
  'result': AgentRunResult(output=AlignementData(score=1, explanation='The tweet indicates that the Trump-Putin talks ended without any agreement, with both the US and Russia blaming each other for the stalled Ukraine peace efforts, showing no progress and continued hostility.')),
  'expected': 2,
  'success': True,
  'error': None},
 {'promt': '@BBCBreaking: Putin and Trump agree to continue talks on U